In [1]:
from agentflow.buffers.segment_tree import SumTree
import agentflow.buffers.segment_tree_c as segment_tree
from agentflow.common.prefix_sum_tree import PrefixSumTree
import numpy as np

## Construct tree objects

In [21]:
n = int(1e6)

x = (np.random.choice(50,size=n)+1).astype(float)
idx = np.arange(n).astype(np.int32)

sumtree = SumTree(n)
sumtree2 = np.zeros(n*2)
sumtree3 = PrefixSumTree(n)

## Set single value

In [22]:
%%timeit

sumtree[3] = 5

2.66 µs ± 19 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
%%timeit

segment_tree.update_tree(3,5,sumtree2)

529 ns ± 7.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [24]:
%%timeit

sumtree3[3] = 5

5.32 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [25]:
2430/544.

4.466911764705882

In [26]:
np.abs(sumtree2 - np.array(sumtree._data)).max()

0.0

## Set 100 values

In [27]:
x0 = (np.random.choice(50,size=100)+1).astype(float)
idx0 = np.random.choice(n,size=100).astype(np.int32)

In [28]:
x0.shape

(100,)

In [29]:
%%timeit

for i,v in zip(idx0,x0):
    sumtree[i] = v

1.85 ms ± 82.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%%timeit

segment_tree.update_tree_multi(idx0,x0,sumtree2)

3.26 µs ± 471 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [31]:
%%timeit

sumtree3[idx0] = x0

8.27 µs ± 91.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [32]:
np.abs(sumtree2 - np.array(sumtree._data)).max()

0.0

In [33]:
np.abs(sumtree3 - np.array(sumtree._data)[n:]).max(),np.abs(sumtree3._sumtree - np.array(sumtree._data)[:n]).max()

(0.0, 0.0)

In [34]:
np.abs(sumtree3 - sumtree2[n:]).max()

0.0

In [47]:
1800/3

600.0

In [48]:
1800/8

225.0

## Set all values

In [37]:
x = (np.random.choice(50,size=n)+1).astype(float)
idx = np.arange(n).astype(np.int32)

In [38]:
x.shape

(1000000,)

In [39]:
%%timeit

for i,v in enumerate(x):
    sumtree[i] = v

4.77 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit

segment_tree.update_tree_multi(idx,x,sumtree2)

13.9 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
%%timeit

sumtree3[idx] = x

16.6 ms ± 944 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
np.abs(sumtree2 - np.array(sumtree._data)).max()

0.0

In [44]:
np.abs(sumtree3 - sumtree2[n:]).max()

0.0

In [45]:
np.abs(sumtree3.view(np.ndarray) - sumtree2[n:]).max()

0.0

In [46]:
np.abs(sumtree3._sumtree - sumtree2[:n]).max()

0.0

In [49]:
5.*1000/15

333.3333333333333

## get prefix sum id

In [50]:
vv = np.arange(0,sum(x)+1,10000)
vv.shape

(2551,)

In [51]:
for v in vv:
    assert sumtree.get_prefix_sum_idx(v) == segment_tree.get_prefix_sum_idx(v,sumtree2)
    
v = sum(x)
assert sumtree.get_prefix_sum_idx(v) == segment_tree.get_prefix_sum_idx(v,sumtree2)

In [52]:
%%timeit

sumtree.get_prefix_sum_idx(30)

8.05 µs ± 421 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [53]:
%%timeit

segment_tree.get_prefix_sum_idx(30,sumtree2)

645 ns ± 30.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [54]:
7.11*1000/590.

12.05084745762712

## multiple get prefix sum id

In [55]:
y = np.random.choice(int(x.sum()),size=100).astype(float)

In [56]:
%%timeit

[sumtree.get_prefix_sum_idx(v) for v in y]

608 µs ± 51.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [57]:
%%timeit

[segment_tree.get_prefix_sum_idx(v,sumtree2) for v in y]

77.9 µs ± 4.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [58]:
%%timeit

output = np.zeros(y.shape,dtype=np.int32)
segment_tree.get_prefix_sum_multi_idx(output,y,sumtree2)

11 µs ± 644 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [59]:
out2 = np.zeros(y.shape,dtype=np.int32)

In [60]:
%%timeit

segment_tree.get_prefix_sum_multi_idx(out2,y,sumtree2)

9.14 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [61]:
600/10.

60.0

In [62]:
%%timeit

sumtree3.get_prefix_sum_id(y)

15.2 µs ± 63.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [63]:
out3 = sumtree3.get_prefix_sum_id(y)

In [64]:
(out2 == out3).mean()

1.0

## Set vs Get (sumtree2)

In [65]:
K = 100
x = (np.random.choice(50,size=K)+1).astype(float)
idx = np.random.choice(n,size=K).astype(np.int32)

In [66]:
%%timeit

segment_tree.update_tree_multi(idx,y,sumtree2)

3.15 µs ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [67]:
idx = np.arange(K).astype(np.int32)
y = np.random.choice(int(x.sum()),size=K).astype(float)

In [68]:
%%timeit

segment_tree.get_prefix_sum_multi_idx(out2,y,sumtree2)

9.51 µs ± 535 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Set vs Get (sumtree3)

In [69]:
K = 100
x = (np.random.choice(50,size=K)+1).astype(float)
idx = np.random.choice(n,size=K).astype(np.int32)

In [70]:
%%timeit

sumtree3[idx] = x

8.73 µs ± 510 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [71]:
idx = np.arange(K).astype(np.int32)
y = np.random.choice(int(x.sum()),size=K).astype(float)

In [72]:
%%timeit

sumtree3.get_prefix_sum_id(y)

14 µs ± 510 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [73]:
%%timeit

sumtree3.sample(K)

45.2 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
